# Representation Learning at the Houses of the Oireachtas in Ireland




Notebook dedicated to run representation learning on the Houses of the Oireachtas' data in Ireland.

This work is based on a Senator Representations' work of the US Congress by Nathaniel Tucker:

https://github.com/knathanieltucker/tf-keras-tutorial/blob/master/SenatorRepresentations.ipynb

The Oireachtas is the legislature of the Republic of Ireland.

The Oireachtas consists of:
- The President of Ireland
- The two houses of the Oireachtas:
    - Dáil Éireann (lower house)
    - Seanad Éireann (upper house)
    
Further info: https://en.wikipedia.org/wiki/Oireachtas

Information regarding the Houses of the Oireachtas is featured on their website, including legislation, and is the copyright of the Houses of the Oireachtas: https://beta.oireachtas.ie/ 

Open Data from the Houses of the Oireachtas can be accessed via:
https://beta.oireachtas.ie/en/open-data/

This includes a link to the open data APIs via a Swagger UI:
https://api.oireachtas.ie/

A vote in the Houses of the Oireachtas is also called a division. We will be looking at this divisions (votings) table for the house and the Seanad (Senate):

In [186]:
import requests # http://docs.python-requests.org/
import json

In [187]:
def get_votings():
    
    votings_data = []
    
    table = "divisions" # votings table
    url = 'https://api.oireachtas.ie/v1/{}'.format(table)
    date_start = '1900-01-01'
    date_end = '2017-12-31'
    chamber_type = 'house' # 'committee' or 'house'
    chamber = 'seanad' # 'seanad' or 'dail'
    
    batch = 500
    skip = 0
    limit = batch
    count = 1 # start
    
    while count > 0:
        
        params = dict(chamber_type=chamber_type,
                      chamber=chamber, 
                      date_start=date_start, 
                      date_end=date_end,
                      skip=skip, 
                      limit=limit)
        r = requests.get(url, params=params)
        
        print r.url
        # print "Status Code:", r.status_code
        # print "Headers:", r.headers
        
        # Add data from this batch
        contents = r.json()
        votings = contents["results"][:batch] # in order no to have duplicates
        votings_data.extend(votings)

        # Update count, more to retrieve?
        count = len(votings)
        print "Number retrieved", count
        # Update skip and limit for query
        skip += batch
        limit += batch
        
    return votings_data

In [147]:
# all_vote_data = get_votings()

https://api.oireachtas.ie/v1/divisions?chamber=seanad&limit=500&chamber_type=house&skip=0&date_end=2017-12-31&date_start=1900-01-01
Number retrieved 500
https://api.oireachtas.ie/v1/divisions?chamber=seanad&limit=1000&chamber_type=house&skip=500&date_end=2017-12-31&date_start=1900-01-01
Number retrieved 500
https://api.oireachtas.ie/v1/divisions?chamber=seanad&limit=1500&chamber_type=house&skip=1000&date_end=2017-12-31&date_start=1900-01-01
Number retrieved 500
https://api.oireachtas.ie/v1/divisions?chamber=seanad&limit=2000&chamber_type=house&skip=1500&date_end=2017-12-31&date_start=1900-01-01
Number retrieved 500
https://api.oireachtas.ie/v1/divisions?chamber=seanad&limit=2500&chamber_type=house&skip=2000&date_end=2017-12-31&date_start=1900-01-01
Number retrieved 500
https://api.oireachtas.ie/v1/divisions?chamber=seanad&limit=3000&chamber_type=house&skip=2500&date_end=2017-12-31&date_start=1900-01-01
Number retrieved 500
https://api.oireachtas.ie/v1/divisions?chamber=seanad&limit=350

In [149]:
# with open('data/congress/Ireland/all_vote_data.json', 'w') as outfile:
#      json.dump(all_vote_data, outfile)

In [3]:
all_vote_data = json.load(open('data/congress/Ireland/all_vote_data.json'))

How many votings?

In [4]:
len(all_vote_data)

3516

Verify there is no duplicates:

In [5]:
def verify_no_voting_duplicates(vote_data):
    votes = []
    duplicates = []
    for vote in vote_data:
        uri = vote["division"]["uri"]
        if uri not in votes:
            # print uri
            votes.append(uri)
        else:
            duplicates.append(uri)
    return len(duplicates)

In [6]:
verify_no_voting_duplicates(all_vote_data)

0

In [7]:
def get_senators(vote_data):
    senators = []
    for vote in vote_data:
        div = vote["division"]
        for outcome, voting in div["tallies"].iteritems():            
            if voting is not None:
                # print voting["showAs"] # Tá (YES), Níl (NO), Staon (Abstention)
                # print len(voting["members"])
                for member in voting["members"]:
                    member = member["member"]
                    #last_name, first_name = member["showAs"].split(",")
                    #first_name = first_name[:-1].strip()
                    #last_name = last_name.strip()
                    member_code = member["memberCode"]
                    # print member_code
                    senators.append(member_code)
    return senators

In [8]:
senators = get_senators(all_vote_data)

In [9]:
senators[3]

u'Jerry-Buttimer.S.2007-07-23'

In [10]:
for v, k in enumerate(set(senators)):
    print v, k

0 Cecilia-Keaveney.D.1996-04-02
1 Déirdre-de-Búrca.S.2007-07-23
2 Alex-White.S.2007-07-23
3 Maria-Byrne.S.2016-04-25
4 Colm-Burke.S.2011-05-25
5 Katherine-Zappone.S.2011-05-25
6 Billy-Lawless.S.2016-04-25
7 Helen-McEntee.D.2013-03-27
8 Aodhán-Ó-Ríordáin.D.2011-03-09
9 Dan-Boyle.D.2002-06-06
10 Gerald-Nash.D.2011-03-09
11 Pat-Moylan.S.1997-09-17
12 John-Halligan.D.2011-03-09
13 Pat-O'Neill.S.2011-05-25
14 Kieran-Phelan.S.2002-09-12
15 Martin-Kenny.D.2016-10-03
16 John-Gilroy.S.2011-05-25
17 Fergal-Browne.S.2002-09-12
18 Colette-Kelleher.S.2016-04-25
19 Neale-Richmond.S.2016-04-25
20 Kate-Walsh.S.2002-09-12
21 John-Whelan.S.2011-05-25
22 John-Browne.D.1982-12-14
23 Frank-Feighan.S.2002-09-12
24 Pearse-Doherty.S.2007-07-23
25 Mary-Ann-O'Brien.S.2011-05-25
26 Jerry-Buttimer.S.2007-07-23
27 Derek-McDowell.D.1992-12-14
28 Niall-Ó-Brolcháin.S.2009-12-14
29 Cyprian-Brady.S.2002-09-12
30 Paschal-Donohoe.S.2007-07-23
31 Alice-Mary-Higgins.S.2016-04-25
32 Paul-Daly.S.2016-04-25
33 Peter-Callanan.

In [11]:
# leave the first two blank for padding and not senators
senator_to_id = { k: v + 2 for v, k in enumerate(set(senators)) }

In [12]:
senator_to_id

{None: 94,
 u'Aidan-Davitt.S.2016-04-25': 60,
 u'Aideen-Hayden.S.2011-05-25': 134,
 u'Alan-Kelly.S.2007-07-23': 67,
 u'Alex-White.S.2007-07-23': 4,
 u'Alice-Mary-Higgins.S.2016-04-25': 33,
 u'Ann-Ormonde.S.1993-02-17': 193,
 u'Aodh\xe1n-\xd3-R\xedord\xe1in.D.2011-03-09': 10,
 u'Averil-Power.S.2011-05-25': 154,
 u'Billy-Lawless.S.2016-04-25': 8,
 u'Brendan-Daly.D.1973-03-14': 128,
 u'Brendan-Kenneally.D.1989-06-29': 76,
 u'Brendan-Ryan.S.1982-05-13': 157,
 u'Brendan-Ryan.S.2007-07-23': 186,
 u'Brian-Hayes.S.1995-12-20': 74,
 u'Brian-\xd3-Domhnaill.S.2007-07-23': 149,
 u'Camillus-Glynn.S.1997-09-17': 84,
 u'Catherine-Ardagh.S.2016-04-25': 85,
 u'Catherine-Noone.S.2011-05-25': 115,
 u'Cecilia-Keaveney.D.1996-04-02': 2,
 u'Chris-Wall.S.2007-06-23': 57,
 u'Ciaran-Cannon.S.2007-07-23': 42,
 u'Colette-Kelleher.S.2016-04-25': 20,
 u'Colm-Burke.S.2011-05-25': 6,
 u'Cyprian-Brady.S.2002-09-12': 31,
 u'C\xe1it-Keane.S.2011-05-25': 56,
 u'Dan-Boyle.D.2002-06-06': 11,
 u"Darragh-O'Brien.D.2007-06-1

In [67]:
def get_senator_id(name):
    name = name.replace(" ", "-").replace(".", "")
    sens = [k for k, v in senator_to_id.iteritems() if k is not None and k.startswith(name)]
    if len(sens) > 0:
        return senator_to_id[sens[0]]
    # print "Bad format for getting the senator ID"
    return 1

In [68]:
print '\xc3\xa1' == str('á')
print str('á').decode('utf-8').encode('utf-8')
print str('á').decode('utf-8').encode('utf-8') == str('\xc3\xa1')
print str('Tá').decode('utf-8').encode('utf-8') == str('T\xc3\xa1')
print str('Níl').decode('utf-8').encode('utf-8') == str('N\xc3\xadl')

True
á
True
True
True


In [69]:
# Sponsors and members against a Bill are specified in a text line where we will extract:
# 2 members that sponsored the bill
# 2 members that opposed to the bill
def get_sponsors(tellers_data):
    
    #print tellers_data
    
    sp_first, sp_second, ag_first, ag_second = None, None, None, None
    
    if ";" in tellers_data:
        
        tellers = tellers_data.split(";")
        
        for teller in tellers:
            
            if teller.strip() == "":
                continue
            if "Tellers:" in teller:
                teller = teller.split("Tellers:")[1].strip()
            elif ":" in teller:
                teller = teller.split(":")[1].strip()

            if "," in teller:
                values = teller.split(",")
            elif ":" in teller:
                values = teller.split(":")
            else:
                #print "Bad format"
                continue
            if len(values) != 2:
                #print "BAD format"
                continue
                
            support, senators = values

            if "Senators" in senators:
                senators = senators.split("Senators")
                senators = senators[-1]
                
            if " and" in senators:
                senators = senators.split(" and")
            elif "and " in senators:
                senators = senators.split("and ")
            elif "an d":
                senators = senators.split("an d")
            else:
                senators = senators.split("and")
            
            first_senator, second_senator = senators[:2]
            first_senator = first_senator.strip()
            second_senator = second_senator.strip()
            support = support.strip()

            if support.encode('utf-8') == str('T\xc3\xa1'):
                sp_first = get_senator_id(first_senator)
                sp_second = get_senator_id(second_senator)
                #print "Sponsors", first_senator, sp_first, second_senator, sp_second
            elif support.encode('utf-8') == str('N\xc3\xadl'):
                ag_first = get_senator_id(first_senator)
                ag_second = get_senator_id(second_senator)
                #print "Against", first_senator, ag_first, second_senator, ag_second
                
    return sp_first, sp_second, ag_first, ag_second 

In [172]:
# Not ALL senators will appear on the votings 
id_to_displayname = {}
for member_code in senators:
    senator_id = senator_to_id[member_code]
    name = member_code
    if name is not None and '.' in name:
        name = name.split('.')[0]
    id_to_displayname[senator_id] = name

In [173]:
id_to_displayname

{2: u'Cecilia-Keaveney',
 3: u'D\xe9irdre-de-B\xfarca',
 4: u'Alex-White',
 5: u'Maria-Byrne',
 6: u'Colm-Burke',
 7: u'Katherine-Zappone',
 8: u'Billy-Lawless',
 9: u'Helen-McEntee',
 10: u'Aodh\xe1n-\xd3-R\xedord\xe1in',
 11: u'Dan-Boyle',
 12: u'Gerald-Nash',
 13: u'Pat-Moylan',
 14: u'John-Halligan',
 15: u"Pat-O'Neill",
 16: u'Kieran-Phelan',
 17: u'Martin-Kenny',
 18: u'John-Gilroy',
 19: u'Fergal-Browne',
 20: u'Colette-Kelleher',
 21: u'Neale-Richmond',
 22: u'Kate-Walsh',
 23: u'John-Whelan',
 24: u'John-Browne',
 25: u'Frank-Feighan',
 26: u'Pearse-Doherty',
 27: u"Mary-Ann-O'Brien",
 28: u'Jerry-Buttimer',
 29: u'Derek-McDowell',
 30: u'Niall-\xd3-Brolch\xe1in',
 31: u'Cyprian-Brady',
 32: u'Paschal-Donohoe',
 33: u'Alice-Mary-Higgins',
 34: u'Paul-Daly',
 35: u'Peter-Callanan',
 36: u'James-Carroll',
 37: u"Darragh-O'Brien",
 38: u'Fidelma-Healy-Eames',
 39: u"Jennifer-Murnane-O'Connor",
 40: u'Larry-Butler',
 41: u"Kate-O'Connell",
 42: u'Ciaran-Cannon',
 43: u'Geraldine-F

In [174]:
senator_vote_data = []

for vote in all_vote_data:
    
    div = vote["division"]
    #outcome = div["outcome"] # Carried, Lost, _
    #print div["date"]
    #print div["isBill"]
    #print div["category"]
    #print div["chamber"]["showAs"]
    #print div["debate"]["showAs"]
    
    # Bill sponsors
    sponsors = get_sponsors(div["tellers"])
    
    if None not in sponsors:
        
        sponsor, cosponsor, against, coagainst = sponsors
        
        tallies = div["tallies"]
        
        if tallies["taVotes"] is not None:
            #print tallies["taVotes"]["showAs"]
            #print len(tallies["taVotes"]["members"])
            for member in tallies["taVotes"]["members"]:
                member = member["member"]
                name = member["showAs"]
                #uri = member["uri"]
                #short = member["memberCode"]
                senator_id = senator_to_id[member["memberCode"]]
                id_to_displayname[senator_id] = name # replace names with utf8 encoding names
                senator_vote_data.append((1, senator_id, sponsor, [cosponsor, against, coagainst])) 
                
        elif tallies["nilVotes"] is not None:
            for member in tallies["nilVotes"]["members"]:
                member = member["member"]
                name = member["showAs"]
                senator_id = senator_to_id[member["memberCode"]]
                id_to_displayname[senator_id] = name
                senator_vote_data.append((0, senator_id, sponsor, [cosponsor, against, coagainst]))     

In [175]:
senator_vote_data

[(1, 6, 191, [118, 102, 93]),
 (1, 123, 191, [118, 102, 93]),
 (1, 159, 191, [118, 102, 93]),
 (1, 28, 191, [118, 102, 93]),
 (1, 5, 191, [118, 102, 93]),
 (1, 87, 191, [118, 102, 93]),
 (1, 147, 191, [118, 102, 93]),
 (1, 34, 191, [118, 102, 93]),
 (1, 60, 191, [118, 102, 93]),
 (1, 25, 191, [118, 102, 93]),
 (1, 91, 191, [118, 102, 93]),
 (1, 163, 191, [118, 102, 93]),
 (1, 196, 191, [118, 102, 93]),
 (1, 20, 191, [118, 102, 93]),
 (1, 130, 191, [118, 102, 93]),
 (1, 191, 191, [118, 102, 93]),
 (1, 140, 191, [118, 102, 93]),
 (1, 81, 191, [118, 102, 93]),
 (1, 39, 191, [118, 102, 93]),
 (1, 166, 191, [118, 102, 93]),
 (1, 72, 191, [118, 102, 93]),
 (1, 145, 191, [118, 102, 93]),
 (1, 118, 191, [118, 102, 93]),
 (1, 100, 191, [118, 102, 93]),
 (1, 168, 191, [118, 102, 93]),
 (1, 21, 191, [118, 102, 93]),
 (1, 177, 191, [118, 102, 93]),
 (1, 73, 191, [118, 102, 93]),
 (1, 85, 85, [165, 191, 118]),
 (1, 158, 85, [165, 191, 118]),
 (1, 165, 85, [165, 191, 118]),
 (1, 105, 85, [165, 191, 

In [176]:
id_to_displayname

{2: u'Keaveney, Cecilia.',
 3: u'de B\xfarca, D\xe9irdre.',
 4: u'White, Alex.',
 5: u'Byrne, Maria.',
 6: u'Burke, Colm.',
 7: u'Zappone, Katherine.',
 8: u'Lawless, Billy.',
 9: u'Helen-McEntee',
 10: u'\xd3 R\xedord\xe1in, Aodh\xe1n.',
 11: u'Boyle, Dan.',
 12: u'Craughwell, Gerard P.',
 13: u'Moylan, Pat.',
 14: u'John-Halligan',
 15: u'O\u2019Neill, Pat.',
 16: u'Phelan, Kieran.',
 17: u"O'Brien, Mary Ann.",
 18: u'Gilroy, John.',
 19: u'Browne, Fergal.',
 20: u'Kelleher, Colette.',
 21: u'Richmond, Neale.',
 22: u'Walsh, Kate.',
 23: u'Whelan, John.',
 24: u'Crown, John.',
 25: u'Feighan, Frank.',
 26: u'Doherty, Pearse.',
 27: u'O\u2019Brien, Mary Ann.',
 28: u'Buttimer, Jerry.',
 29: u'McDowell, Derek.',
 30: u'\xd3 Brolch\xe1in, Niall.',
 31: u'Brady, Cyprian.',
 32: u'Donohoe, Paschal.',
 33: u'Higgins, Alice-Mary.',
 34: u'Daly, Paul.',
 35: u'Callanan, Peter.',
 36: u'Carroll, James.',
 37: u'O\u2019Brien, Darragh.',
 38: u'Healy Eames, Fidelma.',
 39: u"Murnane O'Connor, J

In [177]:
len(senator_vote_data)

59287

~60k examples of (vote, senator voting, sponsor, cosponsor, against, coagainst) tuples is pretty good (we could of course scrape more).

In [160]:
y = [d[0] for d in senator_vote_data]

In [161]:
len(y)

59287

In [162]:
# again we pad
def pad_or_crop(lst, l=10):
    return (lst + [0] * l)[:10]

In [163]:
import numpy as np

x_1 = np.array(map(lambda x: x[1], senator_vote_data))
x_2 = np.array(map(lambda x: x[2], senator_vote_data))
x_3 = np.array(map(lambda x: pad_or_crop(x[3]), senator_vote_data))
x = [x_1, x_2, x_3]

In [164]:
x

[array([  6, 123, 159, ...,  94,  94,  94]),
 array([191, 191, 191, ...,   1,   1,   1]),
 array([[118, 102,  93, ...,   0,   0,   0],
        [118, 102,  93, ...,   0,   0,   0],
        [118, 102,  93, ...,   0,   0,   0],
        ..., 
        [  1,   1,   1, ...,   0,   0,   0],
        [  1,   1,   1, ...,   0,   0,   0],
        [  1,   1,   1, ...,   0,   0,   0]])]

In [165]:
len(x)

3

In [178]:
# we add in padding and unknown senators
id_to_displayname[0] = '<PAD>'
id_to_displayname[1] = '<NOT A SENATOR>'

In [179]:
id_to_displayname

{0: '<PAD>',
 1: '<NOT A SENATOR>',
 2: u'Keaveney, Cecilia.',
 3: u'de B\xfarca, D\xe9irdre.',
 4: u'White, Alex.',
 5: u'Byrne, Maria.',
 6: u'Burke, Colm.',
 7: u'Zappone, Katherine.',
 8: u'Lawless, Billy.',
 9: u'Helen-McEntee',
 10: u'\xd3 R\xedord\xe1in, Aodh\xe1n.',
 11: u'Boyle, Dan.',
 12: u'Craughwell, Gerard P.',
 13: u'Moylan, Pat.',
 14: u'John-Halligan',
 15: u'O\u2019Neill, Pat.',
 16: u'Phelan, Kieran.',
 17: u"O'Brien, Mary Ann.",
 18: u'Gilroy, John.',
 19: u'Browne, Fergal.',
 20: u'Kelleher, Colette.',
 21: u'Richmond, Neale.',
 22: u'Walsh, Kate.',
 23: u'Whelan, John.',
 24: u'Crown, John.',
 25: u'Feighan, Frank.',
 26: u'Doherty, Pearse.',
 27: u'O\u2019Brien, Mary Ann.',
 28: u'Buttimer, Jerry.',
 29: u'McDowell, Derek.',
 30: u'\xd3 Brolch\xe1in, Niall.',
 31: u'Brady, Cyprian.',
 32: u'Donohoe, Paschal.',
 33: u'Higgins, Alice-Mary.',
 34: u'Daly, Paul.',
 35: u'Callanan, Peter.',
 36: u'Carroll, James.',
 37: u'O\u2019Brien, Darragh.',
 38: u'Healy Eames, F

In [180]:
# this gives us how many representations:
len(id_to_displayname)

200

In [181]:
# we again need to write down the metadata
import csv

with open('data/congress/Ireland/senator_metadata.csv', 'wb') as csvfile:
    writer = csv.writer(csvfile, delimiter='\t')
    for key, value in sorted(id_to_displayname.items()):
        writer.writerow([value.encode('utf8')])

In [182]:
# finally we build our model
from keras.layers import concatenate
from keras.layers import Dense, Input, Flatten
from keras.layers import MaxPooling1D, Embedding

embedding_layer = Embedding(len(id_to_displayname), 100)

# train a 1D convnet with global maxpooling
voting = voting_input = Input(shape=(1,), dtype='int32')
voting = embedding_layer(voting)
voting = Dense(32, activation='relu')(voting)
voting = Dense(32, activation='relu')(voting)

sponsor = sponsor_input = Input(shape=(1,), dtype='int32')
sponsor = embedding_layer(sponsor)
sponsor = Dense(32, activation='relu')(sponsor)
sponsor = Dense(32, activation='relu')(sponsor)

cosponsor = cosponsor_input = Input(shape=(10,), dtype='int32')
cosponsor = embedding_layer(cosponsor)
cosponsor = MaxPooling1D(10)(cosponsor)
cosponsor = Dense(32, activation='relu')(cosponsor)
cosponsor = Dense(32, activation='relu')(cosponsor)

combined = concatenate([voting, sponsor, cosponsor])
combined = Dense(32, activation='relu')(combined)
combined = Dense(1, activation='sigmoid')(combined)

In data/congress/Ireland, launch TensorBoard:
    
> davids-air:Ireland dazconap$ tensorboard --logdir=senator_reps/

In [183]:
from keras.models import Model
from keras.callbacks import TensorBoard

model = Model([voting_input, sponsor_input, cosponsor_input], combined)

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])

embedding_metadata = {
    embedding_layer.name: '../senator_metadata.csv'
}

model.fit([x_1, x_2, x_3], np.array(y).reshape(-1, 1, 1),
          batch_size=128,
          epochs=10,
          validation_split=0.2,
          callbacks=[TensorBoard(log_dir='data/congress/Ireland/senator_reps', 
                                 embeddings_freq=1,
                                 embeddings_metadata=embedding_metadata)])

Train on 47429 samples, validate on 11858 samples
Epoch 1/10
47429/47429 [==============================] - 12s 263us/step - loss: 0.0212 - acc: 0.9999 - val_loss: 1.1921e-07 - val_acc: 1.0000
Epoch 2/10
47429/47429 [==============================] - 12s 246us/step - loss: 1.1921e-07 - acc: 1.0000 - val_loss: 1.1921e-07 - val_acc: 1.0000
Epoch 3/10
47429/47429 [==============================] - 13s 264us/step - loss: 1.1921e-07 - acc: 1.0000 - val_loss: 1.1921e-07 - val_acc: 1.0000
Epoch 4/10
47429/47429 [==============================] - 12s 247us/step - loss: 1.1921e-07 - acc: 1.0000 - val_loss: 1.1921e-07 - val_acc: 1.0000
Epoch 5/10
47429/47429 [==============================] - 13s 269us/step - loss: 1.1921e-07 - acc: 1.0000 - val_loss: 1.1921e-07 - val_acc: 1.0000
Epoch 6/10
47429/47429 [==============================] - 12s 246us/step - loss: 1.1921e-07 - acc: 1.0000 - val_loss: 1.1921e-07 - val_acc: 1.0000
Epoch 7/10
47429/47429 [==============================] - 13s 284us/step

Go to TensorBoard:
> http://localhost:6006/#projector
        
In TensorBoard, we can look at the representations created in our model using t-SNE or PCA. A t-SNE analysis using more than a 1.1K iterations divides the senators in two different groups:

![title](data/congress/Ireland/t-SNE.png)